### Función para conectar a la base de datos

In [1]:
import oracledb
from datetime import datetime

usuario = "x9038631"
contraseña = "x9038631"
dsn = oracledb.makedsn("oracle0.ugr.es", 1521, service_name="practbd")

def conectar_bd():
    try:
        conexion = oracledb.connect(user=usuario, password=contraseña, dsn=dsn)
        cursor = conexion.cursor()
        print("Conexión a la base de datos realizada correctamente")
        return conexion, cursor
    except oracledb.Error as e:
        return None

# Subsistema de Inventario

#### RF 1.1. Dar de alta un producto

In [2]:
def dar_alta_producto():
    codigo_producto = input("Ingrese el código del producto (máximo 13 caracteres): ").strip()
    nombre = input("Ingrese el nombre del producto: ").strip()
    descripcion = input("Ingrese una descripción para el producto: ").strip()
    
    precio_compra = -1
    while precio_compra < 0:
        precio_compra = float(input("Ingrese el precio del producto: "))
        if precio_compra < 0:
            print("El precio no puede ser negativo.")
 
    precio_venta = -1
    while precio_venta < 0:
        precio_venta = float(input("Ingrese el precio de venta del producto: "))
        if precio_venta < 0:
            print("El precio de venta no puede ser negativo.")

    stock = -1
    while stock < 0:
        stock = int(input("Ingrese el stock inicial del producto: "))
        if stock < 0:
            print("El stock no puede ser negativo.")

    stock_minimo = -1
    while stock_minimo < 0:
        stock_minimo = int(input("Ingrese el stock mínimo permitido para el producto: "))
        if stock_minimo < 0:
            print("El stock mínimo no puede ser negativo.")

    categoria = input("Ingrese la categoría del producto (opcional): ").strip()

    try:
        conn, cursor = conectar_bd()
        cursor.execute("""
            INSERT INTO Producto (CodigoProducto, Nombre, Descripcion, PrecioCompra, PrecioVenta, Stock, StockMinimo, Categoria) 
            VALUES (:codigo_producto, :nombre, :descripcion, :precio_compra, :precio_venta, :stock, :stock_minimo, :categoria)
        """, [codigo_producto, nombre, descripcion, precio_compra, precio_venta, stock, stock_minimo, categoria])
        conn.commit()
        print(f"Producto '{nombre}' añadido exitosamente.")
    except oracledb.DatabaseError as e:
        conn.rollback()
        print(f"Error al añadir producto: {e}")
    finally:
        cursor.close()
        conn.close()

#### RF 1.2. Modificar el stock de un producto

In [3]:
def modificar_stock_producto():
    codigo_producto = input("Ingrese el código del producto que desea modificar: ").strip()

    nuevo_stock = -1
    while nuevo_stock < 0:
        nuevo_stock = int(input("Ingrese el nuevo valor de stock para este producto: "))
        if nuevo_stock < 0:
            print("El stock no puede ser negativo.")
            
    try:
        conn, cursor = conectar_bd()
 
        cursor.execute("SELECT COUNT(*) FROM Producto WHERE CodigoProducto = :codigo_producto", {"codigo_producto": codigo_producto})
        producto_existe = cursor.fetchone()[0]

        if producto_existe == 0:
            print(f"No se encontró ningún producto con el código: {codigo_producto}.")
            return

        cursor.execute("""
            UPDATE Producto 
            SET Stock = :nuevo_stock
            WHERE CodigoProducto = :codigo_producto
        """, {
            "nuevo_stock": nuevo_stock,
            "codigo_producto": codigo_producto
        })
        conn.commit()
        print(f"El stock del producto con código {codigo_producto} ha sido actualizado exitosamente a {nuevo_stock}.")
    except oracledb.DatabaseError as e:
        conn.rollback()
        print(f"Error al modificar el stock del producto: {e}")
    finally:
        cursor.close()
        conn.close()


#### RF 1.3. Consultar el stock de un producto

In [4]:
def consultar_stock_producto():
    codigo_producto = input("Ingrese el codigo del producto que desea consultar: ").strip()

    try:
        conn, cursor = conectar_bd()

        cursor.execute("""
            SELECT Nombre, Stock 
            FROM Producto 
            WHERE CodigoProducto = :codigo_producto
        """, {"codigo_producto": codigo_producto})
        
        producto = cursor.fetchone()

        if producto:
            nombre, stock = producto
            print(f"El producto '{nombre}' tiene un stock actual de {stock} unidades.")
        else:
            print(f"No se encontro ningun producto con el codigo: {codigo_producto}.")
    except oracledb.DatabaseError as e:
        print(f"Error al consultar el stock del producto: {e}")
    finally:
        cursor.close()
        conn.close()

#### RF 1.4. Consultar Alerta de un Producto

In [5]:
def consultar_alerta_producto():
    codigo_producto = input("Ingrese el código del producto para consultar las alertas: ").strip()
    try:
        conn, cursor = conectar_bd()
    
        cursor.execute("SELECT Nombre FROM Producto WHERE CodigoProducto = :codigo_producto", {"codigo_producto": codigo_producto})
        producto = cursor.fetchone()

        if not producto:
            print(f"No se encontró ningún producto con el código: {codigo_producto}.")
            return

        cursor.execute("""
            SELECT CodigoAlerta, FechaHoraAlerta 
            FROM Alerta 
            WHERE CodigoProducto = :codigo_producto
            ORDER BY FechaHoraAlerta DESC
        """, {"codigo_producto": codigo_producto})

        alertas = cursor.fetchall()

        if alertas:
            print(f"Alertas para el producto '{producto[0]}' (Código: {codigo_producto}):")
            for alerta in alertas:
                print(f" - Alerta ID: {alerta[0]}, Fecha y hora: {alerta[1]}")
        else:
            print(f"No hay alertas registradas para el producto '{producto[0]}' (Código: {codigo_producto}).")
    except oracledb.DatabaseError as e:
        print(f"Error al consultar las alertas del producto: {e}")
    finally:
        cursor.close()
        conn.close()


#### RF 1.5. Dar de baja un producto

In [6]:
def dar_de_baja_producto():
    codigo_producto = input("Ingrese el código del producto que desea dar de baja: ").strip()
    try:
        conn, cursor = conectar_bd()
        
        cursor.execute("SELECT Nombre, Estado FROM Producto WHERE CodigoProducto = :codigo_producto", {"codigo_producto": codigo_producto})
        producto = cursor.fetchone()

        if not producto:
            print(f"No se encontró ningún producto con el código: {codigo_producto}.")
            return

        nombre_producto, estado_actual = producto

        if estado_actual == 0:
            print(f"El producto '{nombre_producto}' (Código: {codigo_producto}) ya está descatalogado.")
            return
        
        cursor.execute("""
            UPDATE Producto 
            SET Estado = 0
            WHERE CodigoProducto = :codigo_producto
        """, {"codigo_producto": codigo_producto})
        conn.commit()

        print(f"El producto '{nombre_producto}' (Código: {codigo_producto}) ha sido dado de baja exitosamente.")
    except oracledb.DatabaseError as e:
        conn.rollback()
        print(f"Error al dar de baja el producto: {e}")
    finally:
        cursor.close()
        conn.close()

# Subsistema de Ventas

#### RF 2.1. Registrar una venta

In [7]:
def registrar_venta():
    """
    Solicita los datos para registrar una nueva venta en la base de datos.
    """
    try:
        conn, cursor = conectar_bd()
        if not conn or not cursor:
            return

        # Generar código de venta
        cursor.execute("SELECT SEQ_VENTA.NEXTVAL FROM DUAL")
        venta_seq = cursor.fetchone()[0]
        codigo_venta = f"V{venta_seq:012}"
        fecha_hora = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Solicitar el cliente
        codigo_cliente = input("Ingrese el código del cliente: ")

        # Insertar la venta
        cursor.execute("""
            INSERT INTO Venta (CodigoVenta, FechaHoraVenta, EstadoVenta, CodigoCliente)
            VALUES (:1, TO_TIMESTAMP(:2, 'YYYY-MM-DD HH24:MI:SS'), 'Finalizado', :3)
        """, [codigo_venta, fecha_hora, codigo_cliente])

        # Registrar detalles de la venta
        while True:
            codigo_producto = input("Código del producto (o 'fin' para terminar): ")
            if codigo_producto.lower() == 'fin':
                break

            cantidad = int(input("Cantidad a vender: "))

            cursor.execute("""
                SELECT PrecioVenta FROM Producto WHERE CodigoProducto = :codigo_producto
            """, [codigo_producto])
            precio_venta = cursor.fetchone()[0]

            # Generar un código único para el detalle
            cursor.execute("SELECT SEQ_DETALLEVENTA.NEXTVAL FROM DUAL")
            detalle_seq = cursor.fetchone()[0]
            codigo_detalle = f"D{detalle_seq:012}"  # Prefijo "D" con un número de 12 dígitos (total 13)
            cursor.execute("""
                INSERT INTO DetalleVenta (CodigoDetalleVenta, CodigoVenta, CodigoProducto, PrecioVentaFinal, Cantidad)
                VALUES (:codigo_detalle, :codigo_venta, :codigo_producto, :precio_venta, :cantidad)
            """, [codigo_detalle, codigo_venta, codigo_producto, precio_venta, cantidad])

        conn.commit()
        print(f"Venta {codigo_venta} registrada correctamente.")
    except Exception as e:
        if conn:
            conn.rollback()
        print("Error registrando la venta:", e)
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()


#### RF 2.2. Generar factura

In [8]:
def generar_factura():
    """
    Genera una factura para una venta específica.
    """
    try:
        conn, cursor = conectar_bd()
        if not conn or not cursor:
            print("No se pudo establecer conexión con la base de datos.")
            return

        # Mostrar las ventas disponibles
        print("\nVentas disponibles:")
        cursor.execute("SELECT CodigoVenta, FechaHoraVenta, EstadoVenta FROM Venta")
        ventas = cursor.fetchall()
        if ventas:
            for v in ventas:
                print(f"Código: {v[0]}, Fecha: {v[1]}, Estado: {v[2]}")
        else:
            print("No hay ventas registradas.")
            return

        # Solicitar el código de la venta
        codigo_venta = input("Ingrese el código de la venta para generar la factura: ")

        # Verificar que la venta exista
        cursor.execute("SELECT * FROM Venta WHERE CodigoVenta = :codigo_venta", [codigo_venta])
        venta = cursor.fetchone()
        if not venta:
            print(f"No se encontró la venta con código {codigo_venta}.")
            return

        # Generar el código de la factura
        codigo_factura = f"F_{codigo_venta}"

        # Insertar en la tabla Factura
        cursor.execute(
            """
            INSERT INTO Factura (CodigoFactura, CodigoVenta, FechaHoraVenta)
            VALUES (:codigo_factura, :codigo_venta, :fecha_hora)
            """,
            {
                'codigo_factura': codigo_factura,
                'codigo_venta': codigo_venta,
                'fecha_hora': venta[1]  # Fecha y hora de la venta
            }
        )

        conn.commit()
        print(f"Factura generada exitosamente con el código {codigo_factura}.")

    except Exception as e:
        if conn:
            conn.rollback()
        print("Error al generar la factura:", e)

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

#### RF 2.3. Aplicar promoción

In [9]:
def aplicar_promocion():
    """
    Aplica una promoción a un producto y actualiza su precio.
    """
    try:
        conn, cursor = conectar_bd()
        if not conn or not cursor:
            print("No se pudo establecer conexión con la base de datos.")
            return
        
        # Solicitar datos de la promoción
        codigo_producto = input("Ingrese el código del producto: ")
        cursor.execute("SELECT * FROM Producto WHERE CodigoProducto = :codigo_producto", [codigo_producto])
        producto = cursor.fetchone()
        if not producto:
            print(f"No se encontró el producto con código {codigo_producto}.")
            return
        
        codigo_promocion = input("Ingrese el código de la promoción: ")
        descuento = float(input("Ingrese el porcentaje de descuento (entre 0 y 1): "))
        
        if not (0 <= descuento <= 1):
            print("El descuento debe estar entre 0 y 1.")
            return
        
        # Insertar en la tabla Promoción
        cursor.execute(
            """
            INSERT INTO Promocion (CodigoPromocion, DescuentoAplicado)
            VALUES (:codigo_promocion, :descuento)
            """,
            {
                'codigo_promocion': codigo_promocion,
                'descuento': descuento
            }
        )
        
        # Insertar en ProductoPromocion
        cursor.execute(
            """
            INSERT INTO ProductoPromocion (CodigoProducto, CodigoPromocion)
            VALUES (:codigo_producto, :codigo_promocion)
            """,
            {
                'codigo_producto': codigo_producto,
                'codigo_promocion': codigo_promocion
            }
        )
        
        # Actualizar precio del producto
        nuevo_precio = producto[4] * (1 - descuento)  # producto[4] es el precio actual
        cursor.execute(
            """
            UPDATE Producto
            SET PrecioVenta = :nuevo_precio
            WHERE CodigoProducto = :codigo_producto
            """,
            {
                'nuevo_precio': nuevo_precio,
                'codigo_producto': codigo_producto
            }
        )
        
        conn.commit()
        print(f"Promoción aplicada. Nuevo precio de {codigo_producto}: {nuevo_precio:.2f}")
    
    except Exception as e:
        if conn:
            conn.rollback()
        print("Error al aplicar la promoción:", e)
    
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()


#### RF 2.4. Consultar historial de ventas

In [10]:
def consultar_transacciones():
    """
    Muestra el registro de todas las transacciones realizadas.
    """
    try:
        conn, cursor = conectar_bd()
        if not conn or not cursor:
            print("No se pudo establecer conexión con la base de datos.")
            return
        
        # Consultar las transacciones
        cursor.execute(
            """
            SELECT V.CodigoVenta, V.FechaHoraVenta, V.EstadoVenta,
                   C.Nombre AS Cliente, DV.CodigoProducto, DV.Cantidad, DV.PrecioVentaFinal
            FROM Venta V
            JOIN Cliente C ON V.CodigoCliente = C.CodigoCliente
            JOIN DetalleVenta DV ON V.CodigoVenta = DV.CodigoVenta
            """
        )
        transacciones = cursor.fetchall()
        if not transacciones:
            print("No se encontraron transacciones registradas.")
            return
        
        # Mostrar las transacciones
        print("\nRegistro de Transacciones de Ventas:")
        print("-------------------------------------------------------------")
        for t in transacciones:
            print(f"Código Venta: {t[0]}, Fecha y Hora: {t[1]}, Estado: {t[2]}")
            print(f"Cliente: {t[3]}, Producto: {t[4]}, Cantidad: {t[5]}, Precio Final: {t[6]:.2f}")
            print("-------------------------------------------------------------")
    
    except Exception as e:
        print("Error al consultar transacciones:", e)
    
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

#### RF 2.5. Gestionar devolución

In [11]:
def gestionar_devolucion():
    """
    Permite gestionar una devolución, marcando la venta como 'Devuelto' y restaurando el stock.
    """
    try:
        conn, cursor = conectar_bd()
        if not conn or not cursor:
            print("No se pudo establecer conexión con la base de datos.")
            return
        
        # Solicitar código de la venta
        codigo_venta = input("Ingrese el código de la venta para gestionar la devolución: ")
        
        # Verificar que la venta exista
        cursor.execute("SELECT * FROM Venta WHERE CodigoVenta = :codigo_venta", [codigo_venta])
        venta = cursor.fetchone()
        if not venta:
            print(f"No se encontró la venta con código {codigo_venta}.")
            return
        
        # Actualizar el estado de la venta
        cursor.execute(
            """
            UPDATE Venta
            SET EstadoVenta = 'Devuelto'
            WHERE CodigoVenta = :codigo_venta
            """,
            {'codigo_venta': codigo_venta}
        )
        
        # Restaurar stock de los productos
        cursor.execute(
            """
            SELECT CodigoProducto, Cantidad
            FROM DetalleVenta
            WHERE CodigoVenta = :codigo_venta
            """,
            {'codigo_venta': codigo_venta}
        )
        detalles = cursor.fetchall()
        
        for d in detalles:
            cursor.execute(
                """
                UPDATE Producto
                SET Stock = Stock + :cantidad
                WHERE CodigoProducto = :codigo_producto
                """,
                {
                    'cantidad': d[1],
                    'codigo_producto': d[0]
                }
            )
        
        conn.commit()
        print(f"Devolución procesada correctamente para la venta {codigo_venta}.")
    
    except Exception as e:
        if conn:
            conn.rollback()
        print("Error al gestionar devolución:", e)
    
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()


# Subsistema de Clientes

#### RF 3.1. Dar de alta un cliente

In [12]:
def alta_cliente():
    aceptado = ""
    while aceptado != "S" and aceptado != "N":
        aceptado = input("A continuación se le solicitarán datos personales, algunos de los cuales son considerados sensibles. \
                         ¿Está seguro de que desea continuar? (S/N): ")
        if aceptado != "S" and aceptado != "N":
            print("Por favor, introduzca una opción válida.")

    if aceptado == "N":
        print("Operación cancelada.")
        return

    codigo = input("Ingrese el código del cliente: ")
    nombre = input("Ingrese el nombre del cliente: ")
    correo = input("Ingrese el correo electrónico del cliente: ")
    telefono = input("Ingrese el teléfono del cliente: ")
    direccion = input("Ingrese la dirección del cliente: ")
    preferencia_contacto = int(input("Ingrese la preferencia de contacto (0: Correo, 1: Teléfono): "))
    
    try:
        conn, cursor = conectar_bd()
        cursor.execute("""
            INSERT INTO Cliente (CodigoCliente, Nombre, Correo, Telefono, Direccion, PreferenciaContacto) 
            VALUES (:codigo, :nombre, :correo, :telefono, :direccion, :preferencia_contacto)
        """, [codigo, nombre, correo, telefono, direccion, preferencia_contacto])
        conn.commit()
        print(f"Cliente {nombre} añadido exitosamente.")
    except oracledb.DatabaseError as e:
        conn.rollback()
        print(f"Error al añadir cliente: {e}")
    finally:
        cursor.close()
        conn.close()

#### RF 3.2. Consultar historial de compras de un cliente

In [13]:
def consultar_historial_ventas():
    codigo_cliente = input("Ingrese el código del cliente: ")
    
    try:
        conn, cursor = conectar_bd()
        cursor.execute("""
            SELECT v.CodigoVenta, v.FechaHoraVenta
            FROM Venta v
            WHERE v.CodigoCliente = :codigo_cliente
        """, [codigo_cliente])
        ventas = cursor.fetchall()

        if not ventas:
            print("No hay ventas asociadas a este cliente.")
            return
        
        print(f"Historial de ventas para el cliente {codigo_cliente}:")
        for venta in ventas:
            print(f"\nVenta {venta[0]} realizada en la fecha y hora {venta[1]}")
            
            cursor.execute("""
                SELECT dv.CodigoDetalleVenta, dv.CodigoProducto, dv.Cantidad, dv.PrecioVentaFinal, p.Nombre
                FROM DetalleVenta dv
                JOIN Producto p ON dv.CodigoProducto = p.CodigoProducto
                WHERE dv.CodigoVenta = :codigo_venta
            """, [venta[0]])
            detalles = cursor.fetchall()
            
            for detalle in detalles:
                print(f"\tDetalle {detalle[0]}:\n\t{detalle[4]} ({detalle[1]})\n\t - Importe: {detalle[3]}€\n\t - Cantidad: {detalle[2]}\n")
    except oracledb.DatabaseError as e:
        print(f"Error al consultar historial: {e}")
    finally:
        cursor.close()
        conn.close()


#### RF 3.3. Editar datos del cliente

In [14]:
def editar_cliente():
    aceptado = ""
    while aceptado != "S" and aceptado != "N":
        aceptado = input("A continuación se le solicitarán datos personales, algunos de los cuales son considerados sensibles. \
                         ¿Está seguro de que desea continuar? (S/N): ")
        if aceptado != "S" and aceptado != "N":
            print("Por favor, introduzca una opción válida.")

    if aceptado == "N":
        print("Operación cancelada.")
        return
    
    codigo = input("Ingrese el código del cliente a editar: ")
    nombre = input("Ingrese el nuevo nombre del cliente: ")
    correo = input("Ingrese el nuevo correo electrónico del cliente: ")
    telefono = input("Ingrese el nuevo teléfono del cliente: ")
    direccion = input("Ingrese la nueva dirección del cliente: ")
    preferencia_contacto = int(input("Ingrese la nueva preferencia de contacto (0: Correo, 1: Teléfono): "))

    try:
        conn, cursor = conectar_bd()  # Asume que conectar_bd() devuelve conexión y cursor
        # Actualizar los datos del cliente
        cursor.execute("""
            UPDATE Cliente 
            SET Nombre = :nombre, Correo = :correo, Telefono = :telefono, 
                Direccion = :direccion, PreferenciaContacto = :preferencia_contacto
            WHERE CodigoCliente = :codigo
        """, [nombre, correo, telefono, direccion, preferencia_contacto, codigo])
        conn.commit()
        print(f"Cliente con código {codigo} actualizado exitosamente.")
    except oracledb.DatabaseError as e:
        conn.rollback()
        print(f"Error al editar cliente: {e}")
    finally:
        cursor.close()
        conn.close()

#### RF 3.4. Enviar notificación a cliente

In [15]:
def enviar_notificacion():
    codigo_cliente = input("Ingrese el código del cliente: ")
    mensaje = input("Ingrese el mensaje para enviar al cliente: ")
    
    try:
        conn, cursor = conectar_bd()

        cursor.execute("""
            DECLARE
                v_codigo_notificacion VARCHAR2(10);
            BEGIN
                -- Insertar en Notificacion y obtener el código generado
                INSERT INTO Notificacion (Mensaje)
                VALUES (:mensaje)
                RETURNING CodigoNotificacion INTO v_codigo_notificacion;

                -- Insertar en ClienteNotificacion utilizando el código recuperado
                INSERT INTO ClienteNotificacion (CodigoCliente, CodigoNotificacion)
                VALUES (:codigo_cliente, v_codigo_notificacion);
            END; 
        """, [mensaje, codigo_cliente])
        conn.commit()
        print(f"Notificación enviada al cliente {codigo_cliente}.")
    except oracledb.DatabaseError as e:
        conn.rollback()
        print(f"Error al enviar notificación: {e}")
    finally:
        cursor.close()
        conn.close()

#### RF 3.5. Dar de baja cliente

In [16]:
def baja_cliente():
    codigo_cliente = input("Ingrese el código del cliente a dar de baja: ")
    try:
        conn, cursor = conectar_bd()
        cursor.execute("""
            UPDATE Cliente 
            SET Estado = 0 
            WHERE CodigoCliente = :codigo_cliente
        """, [codigo_cliente])
        conn.commit()
        print(f"Cliente {codigo_cliente} dado de baja exitosamente.")
    except oracledb.DatabaseError as e:
        conn.rollback()
        print(f"Error al dar de baja cliente: {e}")
    finally:
        cursor.close()
        conn.close()

# Subsistema de Proveedores

#### RF 4.1. Dar de alta un proveedor

In [17]:
def alta_proveedor():
    # Solicitar datos del proveedor
    codigo = input("Ingrese el código del proveedor (10 caracteres): ")
    nombre = input("Ingrese el nombre del proveedor (40 caracteres): ")
    telefono = input("Ingrese el teléfono del proveedor (9-15 dígitos): ")
    correo = input("Ingrese el correo electrónico del proveedor: ")
    direccion = input("Ingrese la dirección social del proveedor (100 caracteres): ")

    try:
        # Conexión a la base de datos
        conn, cursor = conectar_bd()

        # Ejecución de la inserción
        cursor.execute("""
            INSERT INTO Proveedor (CodigoProveedor, Nombre, Telefono, Correo, DireccionSocial, Estado)
            VALUES (:codigo, :nombre, :telefono, :correo, :direccion, 1)
        """, {
            "codigo": codigo,
            "nombre": nombre,
            "telefono": telefono,
            "correo": correo,
            "direccion": direccion
        })

        # Confirmar los cambios
        conn.commit()
        print(f"Proveedor '{nombre}' añadido exitosamente.")
    
    except oracledb.DatabaseError as e:
        # Manejo de errores
        print(f"Error al añadir proveedor: {e}")
    
    finally:
        # Cerrar la conexión
        cursor.close()
        conn.close()

#### RF 4.2. Realizar un pedido.

In [18]:
def añadir_pedido():
    # Pedir al usuario los datos del pedido
    print("Ingrese los datos del pedido:")
    codigo_pedido = input("Código del pedido (20 caracteres): ")
    fecha_solicitud = input("Fecha de solicitud del pedido (YYYY-MM-DD): ")
    codigo_proveedor = input("Código del proveedor (10 caracteres): ")

    try:
        # Conexión a la base de datos
        conn, cursor = conectar_bd()
        print("Conexión a la base de datos establecida.")

        # Insertar el pedido en la base de datos
        cursor.execute("""
            INSERT INTO Pedido (CodigoPedido, FechaSolicitud, EstadoPedido, CodigoProveedor)
            VALUES (:codigo_pedido, TO_DATE(:fecha_solicitud, 'YYYY-MM-DD'), 'Pendiente', :codigo_proveedor)
        """, {
            "codigo_pedido": codigo_pedido,
            "fecha_solicitud": fecha_solicitud,
            "codigo_proveedor": codigo_proveedor
        })
        print("Pedido insertado correctamente.")


        # Registrar detalles de la venta
        while True:
            codigo_producto = input("Código del producto (o 'fin' para terminar): ")
            if codigo_producto.lower() == 'fin':
                break

            cursor.execute("SELECT * FROM Producto WHERE CodigoProducto = :1", [codigo_producto])
            producto = cursor.fetchone()
            if not producto:
                print(f"Producto con código {codigo_producto} no existe.")
                continue

            cantidad = int(input(f"Ingrese la cantidad para {codigo_producto}: "))

            codigo_detalle = input("Código del detalle de pedido")

            cursor.execute("""
                INSERT INTO DetallePedido (CodigoDetallePedido, CodigoPedido, CodigoProducto, Cantidad)
                VALUES (:1, :2, :3, :4)
            """, [codigo_detalle, codigo_pedido, codigo_producto, cantidad])

        # Confirmar los cambios
        print("Realizando commit...")
        conn.commit()
        print("Commit realizado exitosamente.")

    except oracledb.DatabaseError as e:
        # Manejo de errores
        error, = e.args
        print(f"Error al realizar el pedido: {error.message}")
    
    finally:
        # Cerrar la conexión
        if cursor:
            cursor.close()
        if conn:
            conn.close()

#### RF 4.3. Consultar historial de pedidos de un proveedor.

In [19]:
def consultar_historial_compras():
    codigo_proveedor = input("Ingrese el código del proveedor (CIF): ")
    fecha_inicio = input("Ingrese la fecha de inicio de la consulta (YYYY-MM-DD): ")
    fecha_fin = input("Ingrese la fecha de fin de la consulta (YYYY-MM-DD): ")
    
    try:
        # Conexión a la base de datos
        conn, cursor = conectar_bd()  # Asume que conectar_bd() devuelve conexión y cursor
        
        # Obtener los pedidos realizados por el proveedor dentro del rango de fechas
        cursor.execute("""
            SELECT p.CodigoPedido, p.FechaSolicitud
            FROM Pedido p
            WHERE p.CodigoProveedor = :codigo_proveedor
            AND p.FechaSolicitud BETWEEN TO_DATE(:fecha_inicio, 'YYYY-MM-DD') AND TO_DATE(:fecha_fin, 'YYYY-MM-DD')
        """, [codigo_proveedor, fecha_inicio, fecha_fin])
        
        pedidos = cursor.fetchall()

        if not pedidos:
            print("No se encontraron pedidos para este proveedor en el rango de fechas proporcionado.")
            return
        
        # Mostrar los pedidos y sus detalles
        print(f"Historial de compras para el proveedor {codigo_proveedor}:")
        for pedido in pedidos:
            print(f"- Pedido {pedido[0]} realizado el {pedido[1]}")
            
            # Obtener los detalles de cada pedido
            cursor.execute("""
                SELECT dp.CodigoProducto, dp.Cantidad, p.Nombre, p.PrecioCompra
                FROM DetallePedido dp
                JOIN Producto p ON dp.CodigoProducto = p.CodigoProducto
                WHERE dp.CodigoPedido = :codigo_pedido
            """, [pedido[0]])
            detalles = cursor.fetchall()
            
            for detalle in detalles:
                print(f"  - Detalle: Producto {detalle[0]} ({detalle[2]}), Cantidad {detalle[1]}, Precio de compra {detalle[3]} EUR")
    
    except oracledb.DatabaseError as e:
        print(f"Error al consultar el historial de compras: {e}")
    finally:
        # Cerrar la conexión
        cursor.close()
        conn.close()

#### RF 4.4. Cancelar un pedido.

In [20]:
def cancelar_pedido():
    codigo_pedido = input("Ingrese el código del pedido a cancelar: ")
    motivo_cancelacion = input("Ingrese el motivo de la cancelación: ")

    # Obtener la conexión y el cursor de la base de datos
    conn, cursor = conectar_bd()
    
    if conn is None:
        print("No se pudo establecer la conexión a la base de datos.")
        return
    
    try:
        # Verificar si el pedido existe y está en un estado pendiente
        cursor.execute("""
            SELECT EstadoPedido
            FROM Pedido
            WHERE CodigoPedido = :codigo_pedido
        """, [codigo_pedido])
        
        estado_pedido = cursor.fetchone()

        if not estado_pedido:
            print(f"No se encontró el pedido con el código {codigo_pedido}.")
            return

        # Verificar que el estado del pedido sea "Pendiente"
        if estado_pedido[0] != 'Pendiente':
            print("El pedido no se puede cancelar porque ya ha sido completado o cancelado previamente.")
            return

        # Actualizar el estado del pedido a "Cancelado" en la tabla Pedido
        cursor.execute("""
            UPDATE Pedido
            SET EstadoPedido = 'Cancelado'
            WHERE CodigoPedido = :codigo_pedido
        """, [codigo_pedido])
        
        # Insertar el pedido cancelado en la tabla PedidosCancelados
        fecha_cancelacion = datetime.now().strftime('%Y-%m-%d')
        
        # Generar el código de cancelación, asegurándonos de que no exceda los 20 caracteres
        codigo_cancelacion = f"CAN{codigo_pedido[-17:]}"  # Tomar los últimos 17 caracteres del código del pedido
        
        cursor.execute("""
            INSERT INTO PedidosCancelados (CodigoCancelacion, CodigoPedido, FechaCancelacion, MotivoCancelacion)
            VALUES (:codigo_cancelacion, :codigo_pedido, TO_DATE(:fecha_cancelacion, 'YYYY-MM-DD'), :motivo_cancelacion)
        """, {
            "codigo_cancelacion": codigo_cancelacion,
            "codigo_pedido": codigo_pedido,
            "fecha_cancelacion": fecha_cancelacion,
            "motivo_cancelacion": motivo_cancelacion
        })
        
        # Confirmar los cambios
        conn.commit()
        print(f"El pedido {codigo_pedido} ha sido cancelado exitosamente y registrado en la tabla de pedidos cancelados.")

    except oracledb.DatabaseError as e:
        print(f"Error al cancelar el pedido: {e}")

    finally:
        # Cerrar la conexión y el cursor si fueron inicializados
        if cursor:
            cursor.close()
        if conn:
            conn.close()

#### RF 4.5. Marcar un pedido como completado.

In [21]:
def completar_pedido():
    codigo_pedido = input("Ingrese el código del pedido a completar: ")
    
    try:
        conn, cursor = conectar_bd()  # Asume que conectar_bd() devuelve la conexión y el cursor
        if conn is None:
            print("No se pudo establecer la conexión a la base de datos.")
            return
        
        # Verificar si el pedido existe y está en un estado pendiente
        cursor.execute("""
            SELECT EstadoPedido
            FROM Pedido
            WHERE CodigoPedido = :codigo_pedido
        """, [codigo_pedido])
        
        estado_pedido = cursor.fetchone()

        if not estado_pedido:
            print(f"No se encontró el pedido con el código {codigo_pedido}.")
            return

        # Verificar que el estado del pedido sea "Pendiente"
        if estado_pedido[0] != 'Pendiente':
            print("El pedido no se puede completar porque no está en estado pendiente.")
            return
        
        # Actualizar el estado del pedido a "Completado"
        cursor.execute("""
            UPDATE Pedido
            SET EstadoPedido = 'Completado'
            WHERE CodigoPedido = :codigo_pedido
        """, [codigo_pedido])
        
        # Confirmar los cambios
        conn.commit()
        print(f"El pedido {codigo_pedido} ha sido completado exitosamente.")

    except oracledb.DatabaseError as e:
        print(f"Error al completar el pedido: {e}")

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

#### RF 4.6. Dar de baja a un proveedor

In [22]:
def dar_de_baja_proveedor():
    codigo_proveedor = input("Ingrese el código del proveedor a dar de baja: ")
    
    try:
        conn, cursor = conectar_bd()
        
        # Verificar si el proveedor tiene pedidos pendientes
        cursor.execute("""
            SELECT COUNT(*) 
            FROM Pedido 
            WHERE CodigoProveedor = :codigo_proveedor AND EstadoPedido = 'Pendiente'
        """, [codigo_proveedor])
        
        pedidos_pendientes = cursor.fetchone()[0]

        if pedidos_pendientes > 0:
            print("El proveedor tiene pedidos pendientes y no puede ser dado de baja.")
            return

        # Cambiar el estado del proveedor a inactivo (0)
        cursor.execute("""
            UPDATE Proveedor 
            SET Estado = '0' 
            WHERE CodigoProveedor = :codigo_proveedor
        """, [codigo_proveedor])
        
        conn.commit()
        print(f"Proveedor {codigo_proveedor} dado de baja exitosamente.")
    
    except oracledb.DatabaseError as e:
        print(f"Error al dar de baja al proveedor: {e}")
    
    finally:
        cursor.close()
        conn.close()

# Subsistema de Análisis e Informes

#### RF 5.1. Generar informe de ventas en un intervalo

In [23]:
def generar_informe_ventas_por_intervalo():

    # Solicitar fechas de inicio y fin al empleado
    fecha_inicio = input("Ingrese la fecha de inicio (YYYY-MM-DD): ")
    fecha_fin = input("Ingrese la fecha de fin (YYYY-MM-DD): ")

    # Validación: Verificar que la fecha de inicio no sea posterior a la fecha de fin
    if fecha_inicio > fecha_fin:
        print("Error: La fecha de inicio no puede ser posterior a la fecha de fin.")
        return
    conn, cursor = conectar_bd() 
    try:
        # Consulta SQL: Ventas realizadas en el intervalo especificado
        cursor.execute("""
            SELECT 
                p.Nombre AS NombreProducto,
                p.CodigoProducto AS CodigoProducto,
                SUM(dv.Cantidad) AS CantidadTotal
            FROM Venta v
            JOIN DetalleVenta dv ON v.CodigoVenta = dv.CodigoVenta
            JOIN Producto p ON dv.CodigoProducto = p.CodigoProducto
            WHERE v.FechaHoraVenta BETWEEN TO_TIMESTAMP(:1, 'YYYY-MM-DD HH24:MI:SS') 
            AND TO_TIMESTAMP(:2, 'YYYY-MM-DD HH24:MI:SS')
            GROUP BY p.Nombre, p.CodigoProducto
            ORDER BY CantidadTotal DESC
        """, [f"{fecha_inicio} 00:00:00", f"{fecha_fin} 23:59:59"])

        # Obtener los resultados
        resultados = cursor.fetchall()

        # Verificar si hay resultados
        if not resultados:
            print(f"No se encontraron ventas entre {fecha_inicio} y {fecha_fin}.")
            return

        # Separar los datos en listas
        nombres_producto = [fila[0] for fila in resultados]
        codigos_producto = [fila[1] for fila in resultados]
        cantidades = [fila[2] for fila in resultados]

        # Mostrar las listas
        print("\nInforme de Ventas:")
        print(f"Nombres de Producto: {nombres_producto}")
        print(f"Códigos de Producto: {codigos_producto}")
        print(f"Cantidad Total Vendida: {cantidades}")

        print("\nInforme generado exitosamente.")

    except Exception as e:
        print(f"Error al generar el informe: {e}")
   

#### RF 5.2. Generar informe de productos más vendidos

In [24]:
def generar_analisis_productos_mas_vendidos():
        # Solicitar fechas de inicio y fin
    fecha_inicio = input("Ingrese la fecha de inicio (YYYY-MM-DD): ")
    fecha_fin = input("Ingrese la fecha de fin (YYYY-MM-DD): ")

    # Validar las fechas
    if fecha_inicio > fecha_fin:
        print("Error: La fecha de inicio no puede ser posterior a la fecha de fin.")
        return

        # Solicitar límite de productos a mostrar
    limite = int(input("Ingrese el límite de productos a mostrar: "))
    if limite <= 0:
        print("Error: El límite debe ser un número entero positivo.")
        return  
    try:  
        conn, cursor = conectar_bd()
        # Consulta SQL: Obtener los productos más vendidos
        cursor.execute(f"""
            SELECT 
                p.Nombre AS NombreProducto,
                p.CodigoProducto AS CodigoProducto,
                SUM(dv.Cantidad) AS CantidadVendida
            FROM Venta v
            JOIN DetalleVenta dv ON v.CodigoVenta = dv.CodigoVenta
            JOIN Producto p ON dv.CodigoProducto = p.CodigoProducto
            WHERE v.FechaHoraVenta BETWEEN TO_TIMESTAMP(:1, 'YYYY-MM-DD HH24:MI:SS')
            AND TO_TIMESTAMP(:2, 'YYYY-MM-DD HH24:MI:SS')
            GROUP BY p.Nombre, p.CodigoProducto
            HAVING SUM(dv.Cantidad) > 0  -- Garantiza solo productos con ventas
            ORDER BY CantidadVendida DESC
            FETCH FIRST :3 ROWS ONLY
        """, [f"{fecha_inicio} 00:00:00", f"{fecha_fin} 23:59:59", limite])

        # Obtener los resultados
        resultados = cursor.fetchall()

        # Verificar si hay resultados
        if not resultados:
            print(f"No se encontraron productos vendidos entre {fecha_inicio} y {fecha_fin}.")
            return

        # Separar los datos en listas
        nombres_producto = [fila[0] for fila in resultados]
        codigos_producto = [fila[1] for fila in resultados]
        cantidades = [fila[2] for fila in resultados]

        # Mostrar las listas
        print("\nAnálisis de Productos Más Vendidos:")
        print(f"Nombres de Producto: {nombres_producto}")
        print(f"Códigos de Producto: {codigos_producto}")
        print(f"Cantidad Vendida: {cantidades}")
        print("\nAnálisis generado exitosamente.")
    except Exception as e:
         print(f"Error al generar el análisis: {e}")

#### RF 5.3. Generar informe de ingresos para un producto

In [25]:
def generar_analisis_ingresos_producto():
        # Solicitar entradas al usuario
        codigo_producto = input("Ingrese el código del producto: ")
        fecha_inicio = input("Ingrese la fecha de inicio (YYYY-MM-DD): ")
        fecha_fin = input("Ingrese la fecha de fin (YYYY-MM-DD): ")

        # Validar las fechas
        if fecha_inicio > fecha_fin:
            print("Error: La fecha de inicio no puede ser posterior a la fecha de fin.")
            return
        try:
            conn,cursor = conectar_bd()
            
        # Consulta SQL: Obtener ingresos del producto en el intervalo
            cursor.execute("""
                SELECT 
                    p.Nombre AS NombreProducto,
                    p.CodigoProducto AS CodigoProducto,
                    SUM(dv.Cantidad * (dv.PrecioVentaFinal - p.PrecioCompra)) AS Ingresos
                FROM Venta v
                JOIN DetalleVenta dv ON v.CodigoVenta = dv.CodigoVenta
                JOIN Producto p ON dv.CodigoProducto = p.CodigoProducto
                WHERE dv.CodigoProducto = :1
                AND v.FechaHoraVenta BETWEEN TO_TIMESTAMP(:2, 'YYYY-MM-DD HH24:MI:SS')
                AND TO_TIMESTAMP(:3, 'YYYY-MM-DD HH24:MI:SS')
                GROUP BY p.Nombre, p.CodigoProducto
        """, [codigo_producto, f"{fecha_inicio} 00:00:00", f"{fecha_fin} 23:59:59"])

        # Obtener los resultados
            resultado = cursor.fetchone()

        # Verificar si hay datos
            if not resultado:
                    print(f"No se encontraron ingresos para el producto {codigo_producto} entre {fecha_inicio} y {fecha_fin}.")
                    return
    
            # Mostrar los datos
            print("\nAnálisis de Ingresos para el Producto:")
            print(f"Nombre del Producto: {resultado[0]}")
            print(f"Código del Producto: {resultado[1]}")
            print(f"Ingresos Totales: {resultado[2]:.2f}")
    
            print("\nAnálisis generado exitosamente.")

        except Exception as e:
            print(f"Error al generar el análisis: {e}")

#### RF 5.4. Generar informe de pedidos de un intervalo

In [26]:
def generar_informe_pedidos_por_intervalo():
        # Solicitar fechas de inicio y fin al usuario
        fecha_inicio = input("Ingrese la fecha de inicio (YYYY-MM-DD): ")
        fecha_fin = input("Ingrese la fecha de fin (YYYY-MM-DD): ")

        # Validar las fechas
        if fecha_inicio > fecha_fin:
            print("Error: La fecha de inicio no puede ser posterior a la fecha de fin.")
            return
        
        try:
            conn,cursor = conectar_bd()
        # Consulta SQL para obtener los pedidos en el intervalo
            cursor.execute("""
                SELECT 
                p.Nombre AS NombreProducto,
                dp.CodigoProducto AS CodigoProducto,
                SUM(dp.Cantidad) AS CantidadTotal,
                p.PrecioCompra AS PrecioCompra
            FROM Pedido ped
            JOIN DetallePedido dp ON ped.CodigoPedido = dp.CodigoPedido
            JOIN Producto p ON dp.CodigoProducto = p.CodigoProducto
            WHERE ped.FechaSolicitud BETWEEN TO_DATE(:1, 'YYYY-MM-DD') AND TO_DATE(:2, 'YYYY-MM-DD')
            GROUP BY p.Nombre, dp.CodigoProducto, p.PrecioCompra
            ORDER BY CantidadTotal DESC
        """, [fecha_inicio, fecha_fin])

        # Obtener los resultados
            resultados = cursor.fetchall()
    
            # Verificar si hay resultados
            if not resultados:
                print(f"No se encontraron pedidos entre {fecha_inicio} y {fecha_fin}.")
                return
    
            # Separar los datos en listas
            nombres_producto = [fila[0] for fila in resultados]
            codigos_producto = [fila[1] for fila in resultados]
            cantidades = [fila[2] for fila in resultados]
            precios_compra = [fila[3] for fila in resultados]
    
            # Mostrar el informe
            print("\nInforme de Pedidos:")
            print(f"Nombres de Producto: {nombres_producto}")
            print(f"Códigos de Producto: {codigos_producto}")
            print(f"Cantidad Total Comprada: {cantidades}")
            print(f"Precios de Compra: {precios_compra}")
    
            print("\nInforme generado exitosamente.")

        except Exception as e:
            print(f"Error al generar el informe: {e}")


#### RF 5.5. Generar análisis de ventas por categoría

In [27]:
def generar_analisis_ventas_por_familia():
        # Solicitar entradas al usuario
        categoria = input("Ingrese la categoría del producto: ")
        fecha_inicio = input("Ingrese la fecha de inicio (YYYY-MM-DD): ")
        fecha_fin = input("Ingrese la fecha de fin (YYYY-MM-DD): ")

        # Validar las fechas
        if fecha_inicio > fecha_fin:
            print("Error: La fecha de inicio no puede ser posterior a la fecha de fin.")
            return
        try:
            conn,cursor = conectar_bd()
            # Consulta SQL: Obtener las ventas agrupadas por familia de productos
            cursor.execute("""
                SELECT 
                    p.Categoria AS Categoria,
                    SUM(dv.Cantidad) AS CantidadTotal,
                    SUM(dv.Cantidad * dv.PrecioVentaFinal) AS Facturacion
                FROM Venta v
                JOIN DetalleVenta dv ON v.CodigoVenta = dv.CodigoVenta
                JOIN Producto p ON dv.CodigoProducto = p.CodigoProducto
                WHERE p.Categoria = :1
                AND v.FechaHoraVenta BETWEEN TO_TIMESTAMP(:2, 'YYYY-MM-DD HH24:MI:SS')
                AND TO_TIMESTAMP(:3, 'YYYY-MM-DD HH24:MI:SS')
                GROUP BY p.Categoria
            """, [categoria, f"{fecha_inicio} 00:00:00", f"{fecha_fin} 23:59:59"])
    
            # Obtener los resultados
            resultado = cursor.fetchone()
    
            # Verificar si hay datos
            if not resultado:
                print(f"No se encontraron ventas para la categoría '{categoria}' entre {fecha_inicio} y {fecha_fin}.")
                return
    
            # Mostrar los datos
            print("\nAnálisis de Ventas por Familia de Productos:")
            print(f"Categoría: {resultado[0]}")
            print(f"Cantidad Total Vendida: {resultado[1]}")
            print(f"Facturación Total: {resultado[2]:.2f}")
    
            print("\nAnálisis generado exitosamente.")
    
        except Exception as e:
            print(f"Error al generar el análisis: {e}")

## Interfaces

In [28]:
def subsistema_inventario(): 
    while True:
        print("\nSubsistema de Gestión de Inventario")
        print("\t1. Dar de alta un producto")
        print("\t2. Modificar el stock de un producto")
        print("\t3. Consultar el stock de un producto")
        print("\t4. Consultar Alerta Producto")
        print("\t5. Dar de baja un producto (Descatalogar)")
        print("\t6. Volver al menú principal")
        opcion = input("Seleccione la opción deseada: ")

        if opcion == "1":
            dar_alta_producto()
        elif opcion == "2":
            modificar_stock_producto()
        elif opcion == "3":
            consultar_stock_producto()
        elif opcion == "4":
            consultar_alerta_producto()
        elif opcion == "5":
            dar_de_baja_producto()
        elif opcion == "6":
            print("Saliendo del sistema...")
            break
        else:
            print("Opción no válida. Intente de nuevo.")

In [29]:
def subsistema_ventas():
    while True:
        print("\nSubsistema de Gestión de Ventas")
        print("\t1. Realizar una venta")
        print("\t2. Generar una factura de venta")
        print("\t3. Aplicar promoción a un producto")
        print("\t4. Consultar transacciones")
        print("\t5. Gestionar devolución")
        print("\t6. Salir")
        opcion = input("Seleccione la opción deseada: ")

        if opcion == "1":
            registrar_venta()
        elif opcion == "2":
            generar_factura()
        elif opcion == "3":
            aplicar_promocion()
        elif opcion == "4":
            consultar_transacciones()
        elif opcion == "5":
            gestionar_devolucion()
        elif opcion == "6":
            print("Saliendo del subsistema...")
            break
        else:
            print("Opción no válida. Intente de nuevo.")

In [30]:
def subsistema_clientes():
    while True:
        print("\nSubsistema de Gestión de Clientes")
        print("1. Dar de alta un cliente")
        print("2. Consultar historial de ventas de un cliente")
        print("3. Editar datos de un cliente")
        print("4. Enviar notificación a un cliente")
        print("5. Dar de baja un cliente")
        print("6. Salir")
        opcion = input("Seleccione una opción: ")

        if opcion == "1":
            alta_cliente()
        elif opcion == "2":
            consultar_historial_ventas()
        elif opcion == "3":
            editar_cliente()
        elif opcion == "4":
            enviar_notificacion()
        elif opcion == "5":
            baja_cliente()
        elif opcion == "6":
            print("Saliendo del subsistema...")
            break
        else:
            print("Opción no válida. Intente de nuevo.")

In [31]:
def subsistema_proveedores():
    while True:
        print("\nSubsistema de Gestión de Proveedores")
        print("1. Dar de alta un proveedor")
        print("2. Añadir un pedido")
        print("3. Consultar historial de compras")
        print("4. Cancelar un pedido")
        print("5. Completar un pedido")
        print("6. Dar de baja un proveedor")
        print("7. Salir")
        opcion = input("Seleccione una opción: ")

        if opcion == "1":
            alta_proveedor()
        elif opcion == "2":
            añadir_pedido()
        elif opcion == "3":
            consultar_historial_compras()
        elif opcion == "4":
            cancelar_pedido()
        elif opcion == "5":
            completar_pedido()
        elif opcion == "6":
            dar_de_baja_proveedor()
        elif opcion == "7":
            print("Saliendo del subsistema...")
            break
        else:
            print("Opción no válida. Intente de nuevo.")

In [32]:
def subsistema_analisis():
    while True:
        print("\nSubsistema de Análisis de Datos")
        print("1. Generar informe de ventas por intervalo")
        print("2. Generar análisis de productos más vendidos")
        print("3. Generar análisis de ingresos por producto")
        print("4. Generar informe de pedidos por intervalo")
        print("5. Generar análisis de ventas por familia")
        print("6. Salir")
        opcion = input("Seleccione una opción: ")

        if opcion == "1":
            generar_informe_ventas_por_intervalo()
        elif opcion == "2":
            generar_analisis_productos_mas_vendidos()
        elif opcion == "3":
            generar_analisis_ingresos_producto()
        elif opcion == "4":
            generar_informe_pedidos_por_intervalo()
        elif opcion == "5":
            generar_analisis_ventas_por_familia()
        elif opcion == "6":
            print("Saliendo del subsistema...")
            break
        else:
            print("Opción no válida. Intente de nuevo.")

#### Interfaz general

In [34]:
print("Bienvenido al sistema de gestión de la Tienda Tradicional LA GLORIA. Su licorería de Confianza")
while True:
    print("\nMenú Principal")
    print("1. Subsistema de Inventario")
    print("2. Subsistema de Ventas")
    print("3. Subsistema de Clientes")
    print("4. Subsistema de Proveedores")
    print("5. Subsistema de Análisis de Datos")
    print("6. Salir")
    opcion = input("Seleccione una opción: ")

    if opcion == "1":
        subsistema_inventario()
    elif opcion == "2":
        subsistema_ventas()
    elif opcion == "3":
        subsistema_clientes()
    elif opcion == "4":
        subsistema_proveedores()
    elif opcion == "5":
        subsistema_analisis()
    elif opcion == "6":
        print("Saliendo del sistema...")
        break
    else:
        print("Opción no válida. Intente de nuevo.")

Bienvenido al sistema de gestión de la Tienda Tradicional LA GLORIA. Su licorería de Confianza

Menú Principal
1. Subsistema de Inventario
2. Subsistema de Ventas
3. Subsistema de Clientes
4. Subsistema de Proveedores
5. Subsistema de Análisis de Datos
6. Salir

Subsistema de Gestión de Clientes
1. Dar de alta un cliente
2. Consultar historial de ventas de un cliente
3. Editar datos de un cliente
4. Enviar notificación a un cliente
5. Dar de baja un cliente
6. Salir
Operación cancelada.

Subsistema de Gestión de Clientes
1. Dar de alta un cliente
2. Consultar historial de ventas de un cliente
3. Editar datos de un cliente
4. Enviar notificación a un cliente
5. Dar de baja un cliente
6. Salir
Conexión a la base de datos realizada correctamente
Cliente Maria añadido exitosamente.

Subsistema de Gestión de Clientes
1. Dar de alta un cliente
2. Consultar historial de ventas de un cliente
3. Editar datos de un cliente
4. Enviar notificación a un cliente
5. Dar de baja un cliente
6. Salir
Co